# Section: Encrypted Deep Learning

- Lesson: Reviewing Additive Secret Sharing
- Lesson: Encrypted Subtraction and Public/Scalar Multiplication
- Lesson: Encrypted Computation in PySyft
- Project: Build an Encrypted Database
- Lesson: Encrypted Deep Learning in PyTorch
- Lesson: Encrypted Deep Learning in Keras
- Final Project

# Lesson: Reviewing Additive Secret Sharing

_For more great information about SMPC protocols like this one, visit https://mortendahl.github.io. With permission, Morten's work directly inspired this first teaching segment._

In [1]:
import random
import numpy as np

BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 29397334547516724707044527778036574441
PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

def encrypt(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def decrypt(sharing):
    return sum(sharing) % Q

def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [2]:
x = encrypt(encode(5.5))
x

[8245513768159383254698169643196350547,
 20399337600086422537581259005941823591,
 752483179270918914765099129448400303]

In [3]:
y = encrypt(encode(2.3))
y

[3934345076206777396986544152122134220,
 7009198289119970925074661682929889662,
 18453791182189976384983321943214550558]

In [4]:
z = add(x,y)
z

(12179858844366160651684713795318484767,
 27408535889206393462655920688871713253,
 19206274361460895299748421072662950861)

In [5]:
decode(decrypt(z))

7.79999999

# Lesson: Encrypted Subtraction and Public/Scalar Multiplication

In [6]:
field = 23740629843760239486723

In [7]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [8]:
(bob_x_share + alices_x_share) % field

5

In [9]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [10]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

4

In [11]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [12]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [13]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [14]:
(bob_z_share + alice_z_share) % field

4

In [15]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [16]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [17]:
bob_x_share + alice_x_share

15

In [18]:
bob_y_share + alice_y_share

11

In [19]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

3

In [20]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [21]:
x = encrypt(encode(5.5))
x

[2053891933962370156133084892960162742,
 14843316703295283893229632553442502445,
 12500125910259070657681810332183909254]

In [22]:
z = imul(x, 3)

In [23]:
decode(decrypt(z))

16.5

# Lesson: Encrypted Computation in PySyft

In [26]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0812 14:17:35.020856 139796382177088 hook.py:98] Torch was already hooked... skipping hooking process


In [27]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [28]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [29]:
x = x.share(bob, alice, crypto_provider=secure_worker)

In [30]:
y = y.share(bob, alice, crypto_provider=secure_worker)

In [31]:
z = x + y
z.get()

tensor([3, 1, 4, 4])

In [32]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [33]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [34]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [35]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [36]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [37]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [38]:
z = x + y
z.get().float_precision()

tensor([3., 1., 4., 4.])

In [39]:
z = x - y
z.get().float_precision()

tensor([-1.,  3.,  2.,  4.])

In [40]:
z = x * y
z.get().float_precision()

tensor([ 2., -2.,  3.,  0.])

In [41]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [42]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [43]:
z = x == y
z.get().float_precision()

tensor([0., 0., 0., 0.])

# Project: Build an Encrypted Database

In [44]:
# try this project here!

In [45]:
import string

In [46]:
char2index = {}
index2char = {}

In [47]:
#char2index

In [48]:
for i, char in enumerate(" " + string.ascii_lowercase + '0123456789' + string.punctuation):
    char2index[char] = i
    index2char[i] = char

In [49]:
def string2values(str_input, max_len=8):
    str_input = str_input[:max_len].lower()
    
    #pad strings shorter than max len
    if(len(str_input) < max_len):
        str_input = str_input + '.' * (max_len - len(str_input))
    
    values = list()
    for char in str_input:
        values.append(char2index[char])
        
    return th.tensor(values).long()
    


In [50]:
str_input = 'Hellddddddoddd'
max_len = 8
string2values(str_input, max_len)

tensor([ 8,  5, 12, 12,  4,  4,  4,  4])

In [51]:
def one_hot(index, length):
    vect = th.zeros(length).long()
    vect[index] = 1
    return vect

In [52]:
one_hot(0, 8)

tensor([1, 0, 0, 0, 0, 0, 0, 0])

In [53]:
one_hot(char2index['p'],len(index2char))

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [54]:
def string2one_hot_matrix(str_input, max_len=8):
    str_input = str_input[:max_len].lower()
    
    #pad strings shorter than max len
    if(len(str_input) < max_len):
        str_input = str_input + '.' * (max_len - len(str_input))
    
    char_vectors = list()
    for char in str_input:
        char_v = one_hot(char2index[char], len(index2char)).unsqueeze(0)
        char_vectors.append(char_v)
    
    return th.cat(char_vectors, dim=0)

def values2string(input_values):
    s = ''
    for value in input_values:
        s += index2char.get(int(value))
    return s

In [55]:
str_a = string2one_hot_matrix('Hello')
str_b = string2one_hot_matrix('Hello')
str_c = string2one_hot_matrix('Hdddello')

In [56]:
(str_a * str_b).sum(1)

tensor([1, 1, 1, 1, 1, 1, 1, 1])

In [57]:
(str_a * str_c).sum(1)

tensor([1, 0, 0, 0, 0, 0, 0, 0])

In [58]:
def string_equals(str_a, str_b):
    vect = (str_a * str_b).sum(1)
    x = vect[0]
    
    for i in range(vect.shape[0] - 1):
        x = x * vect[i * 1]
    
    return x

In [59]:
keys = list()
values = list()
keys.append(string2one_hot_matrix('key1'))
values.append(string2values('value1'))

keys.append(string2one_hot_matrix('key2'))
values.append(string2values('value2'))

In [60]:
query_str = 'key1'

In [61]:
query_matrix = string2one_hot_matrix(query_str)

In [62]:
query_str = 'key1'
def query(query_str):

    key_matches = list()
    for key in keys:
        key_match = string_equals(key, query_matrix)
        key_matches.append(key_match)

    results = values[0] * key_matches[0]
    for i in range(len(values) - 1):
        results += values[i+1] * key_matches[i+1]

    return values2string(results).replace('.', '')

In [63]:
query(query_str)

'value1'

In [64]:
query('key2')

'value1'

In [65]:
values[1] * keys_matches[0]

NameError: name 'keys_matches' is not defined

In [66]:
class EncryptedDB():
    
    def __init__(self, *owners, max_key_len=8, max_val_len=8):
        self.max_key_len = 8
        self.max_val_len = 8
        
        self.keys = list()
        self.values = list()
        self.owners = owners
        
    def add_entry(self, key, value):
        key = string2one_hot_matrix(key)
        key = key.share(*self.owners)
        self.keys.append(key)
        
        value = string2values(value)
        value = value.share(*self.owners)
        self.values.append(value)

    
    def query(self, query_str):
        query_matrix = string2one_hot_matrix(query_str)
        query_matrix = query_matrix.share(*self.owners)

        key_matches = list()
        for key in self.keys:
            key_match = string_equals(key, query_matrix)
            key_matches.append(key_match)

        results = self.values[0] * key_matches[0]
        for i in range(len(self.values) - 1):
            results += self.values[i+1] * key_matches[i+1]
        
        result = result.get()
        
        return values2string(results).replace('.', '')

    

In [67]:
db = EncryptedDB(bob, alice, secure_worker)


In [68]:
db.add_entry('key1', 'value1')
db.add_entry('key2', 'value2')
db.add_entry('key3', 'value3')
db.add_entry('key4', 'value4')

In [69]:
db.values

[(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:91292096930 -> bob:23218834240]
 	-> (Wrapper)>[PointerTensor | me:32705028404 -> alice:71327827396]
 	-> (Wrapper)>[PointerTensor | me:63407421230 -> secure_worker:92429599688]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:95946027945 -> bob:26009368018]
 	-> (Wrapper)>[PointerTensor | me:13317218028 -> alice:12765703847]
 	-> (Wrapper)>[PointerTensor | me:73868251764 -> secure_worker:89831305040]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:66792424939 -> bob:74557086648]
 	-> (Wrapper)>[PointerTensor | me:32984583317 -> alice:69130733903]
 	-> (Wrapper)>[PointerTensor | me:74738995763 -> secure_worker:78251860698]
 	*crypto provider: me*, (Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:37314211009 -> bob:90160209915]
 	-> (Wrapper)>[PointerTensor | me:21473103784 -> alice:49050000178]
 	-> (Wrapper

# Lesson: Encrypted Deep Learning in PyTorch

### Train a Model

In [70]:
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

tensor(2.2706)
tensor(8.3722)
tensor(11.9019)
tensor(1.0825)
tensor(0.9673)
tensor(0.9335)
tensor(0.8958)
tensor(0.8487)
tensor(0.7895)
tensor(0.7154)
tensor(0.6218)
tensor(0.5171)
tensor(0.4111)
tensor(0.3123)
tensor(0.2283)
tensor(0.1707)
tensor(0.1288)
tensor(0.0924)
tensor(0.0705)
tensor(0.0471)


In [71]:
model(data)

tensor([[ 0.1201],
        [-0.0175],
        [ 0.9627],
        [ 0.8613]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [72]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [73]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:95512844910 -> alice:56290586331]
 	-> (Wrapper)>[PointerTensor | me:66209822896 -> bob:44871602368]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:86536016672 -> alice:2701016318]
 	-> (Wrapper)>[PointerTensor | me:66603561764 -> bob:97094583120]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:12185446997 -> alice:24415015931]
 	-> (Wrapper)>[PointerTensor | me:43791548341 -> bob:60423852363]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:64103138224 -> alice:95516471815]
 	-> (Wrapper)>[PointerTensor | me:1404661003 -> bob:6151853296

In [74]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [75]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:28777322496 -> alice:93538643144]
	-> (Wrapper)>[PointerTensor | me:76134377797 -> bob:80381099102]
	*crypto provider: secure_worker*

In [76]:
encrypted_prediction = encrypted_model(encrypted_data)

In [77]:
encrypted_prediction.get().float_precision()

tensor([[ 0.1210],
        [-0.0170],
        [ 0.9630],
        [ 0.8610]])

# Lesson: Encrypted Deep Learning in Keras


## Step 1: Public Training

Welcome to this tutorial! In the following notebooks you will learn how to provide private predictions. By private predictions, we mean that the data is constantly encrypted throughout the entire process. At no point is the user sharing raw data, only encrypted (that is, secret shared) data. In order to provide these private predictions, Syft Keras uses a library called [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted) under the hood. TF Encrypted combines cutting-edge cryptographic and machine learning techniques, but you don't have to worry about this and can focus on your machine learning application.

You can start serving private predictions with only three steps:
- **Step 1**: train your model with normal Keras.
- **Step 2**: secure and serve your machine learning model (server).
- **Step 3**: query the secured model to receive private predictions (client). 

Alright, let's go through these three steps so you can deploy impactful machine learning services without sacrificing user privacy or model security.

Huge shoutout to the Dropout Labs ([@dropoutlabs](https://twitter.com/dropoutlabs)) and TF Encrypted ([@tf_encrypted](https://twitter.com/tf_encrypted)) teams for their great work which makes this demo possible, especially: Jason Mancuso ([@jvmancuso](https://twitter.com/jvmancuso)), Yann Dupis ([@YannDupis](https://twitter.com/YannDupis)), and Morten Dahl ([@mortendahlcs](https://github.com/mortendahlcs)). 

_Demo Ref: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials_

## Train Your Model in Keras

To use privacy-preserving machine learning techniques for your projects you should not have to learn a new machine learning framework. If you have basic [Keras](https://keras.io/) knowledge, you can start using these techniques with Syft Keras. If you have never used Keras before, you can learn a bit more about it through the [Keras documentation](https://keras.io). 

Before serving private predictions, the first step is to train your model with normal Keras. As an example, we will train a model to classify handwritten digits. To train this model we will use the canonical [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

We borrow [this example](https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py) from the reference Keras repository.  To train your classification model, you just run the cell below.

In [78]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, AveragePooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()

model.add(Conv2D(10, (3, 3), input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

11493376/11490434 [==============================] - 2s 0us/step


W0812 14:30:27.380456 139796382177088 deprecation.py:506] From /home/gabriel/miniconda3/envs/pytorch/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 5s 81us/sample - loss: 2.3034 - acc: 0.0949 - val_loss: 2.3027 - val_acc: 0.0981
Epoch 2/2
60000/60000 [==============================] - 5s 79us/sample - loss: 2.3022 - acc: 0.0988 - val_loss: 2.3016 - val_acc: 0.1024
Test loss: 2.3015579902648926
Test accuracy: 0.1024


In [80]:
## Save your model's weights for future private prediction
model.save('short-conv-mnist.h5')

## Step 2: Load and Serve the Model

Now that you have a trained model with normal Keras, you are ready to serve some private predictions. We can do that using Syft Keras.

To secure and serve this model, we will need three TFEWorkers (servers). This is because TF Encrypted under the hood uses an encryption technique called [multi-party computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).

We'll define a Syft Keras model like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run `hook = sy.KerasHook(tf.keras)`. This will add three important new methods to the Keras Sequential class:
 - `share`: will secure your model via secret sharing; by default, it will use the SecureNN protocol from TF Encrypted to secret share your model between each of the three TFEWorkers. Most importantly, this will add the capability of providing predictions on encrypted data.
 - `serve`: this function will launch a serving queue, so that the TFEWorkers can can accept prediction requests on the secured model from external clients.
 - `shutdown_workers`: once you are done providing private predictions, you can shut down your model by running this function. It will direct you to shutdown the server processes manually if you've opted to manually manage each worker.

If you want learn more about MPC, you can read this excellent [blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [85]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

## Model

As you can see, we define almost the exact same model as before, except we provide a `batch_input_shape`. This allows TF Encrypted to better optimize the secure computations via predefined tensor shapes. For this MNIST demo, we'll send input data with the shape of (1, 28, 28, 1). 
We also return the logit instead of softmax because this operation is complex to perform using MPC, and we don't need it to serve prediction requests.

In [82]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=input_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(num_classes, name="logit"))

### Load Pre-trained Weights

With `load_weights` you can easily load the weights you have saved previously after training your model.

In [83]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

## Step 3: Setup Your Worker Connectors

Let's now connect to the TFEWorkers (`alice`, `bob`, and `carol`) required by TF Encrypted to perform private predictions. For each TFEWorker, you just have to specify a host. We then make combine these workers in a cluster.

These workers run a [TensorFlow server](https://www.tensorflow.org/api_docs/python/tf/distribute/Server), which you can either manage manually (`AUTO = False`) or ask the workers to manage for you (`AUTO = True`). If choosing to manually manage them, you will be instructed to execute a terminal command on each worker's host device after calling `model.share()` below.  If all workers are hosted on a single device (e.g. `localhost`), you can choose to have Syft automatically manage the worker's TensorFlow server.

In [91]:
from syft import TFECluster
AUTO = False

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

cluster = sy.TFECluster(alice, bob, carol)
cluster.start()

ImportError: cannot import name 'TFECluster' from 'syft' (/home/gabriel/miniconda3/envs/pytorch/lib/python3.7/site-packages/syft/__init__.py)

## Step 4: Launch 3 Servers

If you have chosen to manually control the workers (i.e. `AUTO = False`) then you now need to launch 3 servers. Look for the exact commands to run in the info messages printed above. You are looking for the following, where the `...` are actual file paths:

- `python -m tf_encrypted.player --config ... server0`
- `python -m tf_encrypted.player --config ... server1`
- `python -m tf_encrypted.player --config ... server2`

## Step 5: Split the Model Into Shares

Thanks to `sy.KerasHook(tf.keras)` you can call the `share` method to transform your model into a TF Encrypted Keras model.

If you have asked to manually manage servers above then this step will not complete until they have all been launched. Note that your firewall may ask for Python to accept incoming connection.

In [87]:
model.share(cluster)

NameError: name 'cluster' is not defined

## Step 6: Serve the Model

Perfect! Now by calling `model.serve`, your model is ready to provide some private predictions. You can set `num_requests` to set a limit on the number of predictions requests served by the model; if not specified then the model will be served until interrupted.

In [ ]:
model.serve(num_requests=3)

## Step 7: Run the Client

At this point open up and run the companion notebook: Section 4b - Encrytped Keras Client

## Step 8: Shutdown the Servers

Once your request limit above, the model will no longer be available for serving requests, but it's still secret shared between the three workers above. You can kill the workers by executing the cell below.

**Congratulations** on finishing Part 12: Secure Classification with Syft Keras and TFE!

In [ ]:
model.stop()
cluster.stop()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))

# Keystone Project - Mix and Match What You've Learned

Description: Take two of the concepts you've learned about in this course (Encrypted Computation, Federated Learning, Differential Privacy) and combine them for a use case of your own design. Extra credit if you can get your demo working with [WebSocketWorkers](https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials/advanced/websockets-example-MNIST) instead of VirtualWorkers! Then take your demo or example application, write a blogpost, and share that blogpost in #general-discussion on OpenMined's slack!!!

Inspiration:
- This Course's Code: https://github.com/Udacity/private-ai
- OpenMined's Tutorials: https://github.com/OpenMined/PySyft/tree/dev/examples/tutorials
- OpenMined's Blog: https://blog.openmined.org

### Ideas for the project
1) One way to encrypt a ML/Ai model
We are going to try to use the library phe to do encryption with the method based on Paillier partially homomorphic encryption.
More information on the links:
* https://giorgiop.github.io/posts/2017/07/13/distributed-machine-learning-and-partially-homomorphic-encryption-1/
* https://ai.googleblog.com/2017/04/federated-learning-collaborative.html

In [113]:
import phe as paillier
import torch as th
import numpy
pubkey, privkey = paillier.generate_paillier_keypair(n_length=1024)

In [109]:
pubkey

<PaillierPublicKey 64f5e14b34>

In [110]:
privkey

<PaillierPrivateKey for <PaillierPublicKey 64f5e14b34>>

In [104]:
secret_numbers = [3.141592653, 300, -4.6e-12]
encrypted_numbers = [pubkey.encrypt(x) for x in secret_numbers]
[privkey.decrypt(x) for x in encrypted_numbers]

[3.141592653, 300, -4.6e-12]

In [111]:
encrypted_numbers

In [106]:
pubkey

<PaillierPublicKey 64f5e14b34>

In [107]:
print(encrypted_numbers[0])

In [108]:
print(encrypted_numbers[0].ciphertext())

65969064893716410820261275101629319157094866904622149596717458632136892028035573860431844951465276372369868364913739261938403712945695007817086764208993656608824656925366153715139714156943425838768592854116615149062449784829025561414424141203360491349313244219243012281359744732607976393516125847954892731808


### Encrypt pytorch tensors and numpy vectors

In [116]:
vec = np.array([1,1,2,3,5,8])
tens = th.tensor(vec)

In [127]:
#encrypted_numbers = [pubkey.encrypt(x) for x in vec]

In [126]:
#for x in vec:
#    print(pubkey.encrypt(x))

### Follow the example from Giorgi Patrini article

In [129]:
import numpy as np
from sklearn.datasets import load_diabetes

import phe as paillier

seed = 42
np.random.seed(seed)

In [130]:
def get_data(n_clients):

    diabetes = load_diabetes()
    y = diabetes.target
    X = diabetes.data

    # Add constant to emulate intercept
    X = np.c_[X, np.ones(X.shape[0])]

    # The features are already preprocessed
    # Shuffle
    perm = np.random.permutation(X.shape[0])
    X, y = X[perm, :], y[perm]

    # Select test at random
    test_size = 50
    test_idx = np.random.choice(X.shape[0], size=test_size, replace=False)
    train_idx = np.ones(X.shape[0], dtype=bool)
    train_idx[test_idx] = False
    X_test, y_test = X[test_idx, :], y[test_idx]
    X_train, y_train = X[train_idx, :], y[train_idx]

    # Split train among multiple clients.
    # The selection is not at random. We simulate the fact that each client
    # sees a potentially very different sample of patients.
    X, y = [], []
    step = int(X_train.shape[0] / n_clients)
    for c in range(n_clients):
        X.append(X_train[step * c: step * (c + 1), :])
        y.append(y_train[step * c: step * (c + 1)])

    return X, y, X_test, y_test

In [ ]:
###